# Bias Mitigation Experiments - Bank Marketing

## Table of Contents

<!-- To return to the table of contents, click on the number at any major section heading.

* [1. Prepare COMPAS data](#1.-Prepare-COMPAS-data)

* [2. Training Baseline Models](#2.-Training-Baseline-Models)
    * [2.1 Logistic Regression](#2.1-Learning-a-Logistic-Regression-(LR)-classifier-on-original-data)
    * [2.2 Random Forest](#2.2-Learning-a-Random-Forest-(RF)-classifier-on-original-data)

* [3. Bias mitigation using pre-processing technique](#3.-Bias-mitigation-using-pre-processing-technique)

   * [3.1 Reweighing](#3.1-Reweighing)
       * [3.1.1 RW on Logistic Regression](#3.1.1.-Learning-a-Logistic-Regression-(LR)-classifier-on-data-transformed-by-reweighing)
       * [3.1.2 RW on Random Forest](#3.1.2.-Learning-a-Random-Forest-(RF)-classifier-on-data-transformed-by-reweighing)
   * [3.2 Disparate impact remover ](#3.2-Disparate-impact-remover)
       * [3.2.1 DIR on Logistic Regression](#3.2.1-Learning-a-Logistic-Regression-(LR)-classifier-on-data-transformed-by-DIR)
       * [3.2.2 DIR on Random Forest](#3.2.2-Learning-a-Random-Forest-(RF)-classifier-on-data-transformed-by-DIR)
  
* [4. Bias mitigation using in-processing technique](#4.-Bias-mitigation-using-in-processing-technique)
    * [4.1 Prejudice Remover (PR)](#4.1-Prejudice-Remover-(PR))
    * [4.2 Adversarial Debiasing](#4.2-Adversarial-Debiasing)

* [5. Bias mitigation using post-processing technique](#5.-Bias-mitigation-using-post-processing-technique)
    * [5.1 Reject option classification (ROC)](#5.1-Reject-option-classification-(RejOpCl))
        * [5.1.1 ROC on Logistic Regression](#5.1.1-Reject-option-classification-on-Logistic-Regression)
        * [5.1.2 ROC on Random Forest](#5.1.2-Reject-option-classification-on-Random-Forest)
    * [5.2 Equalized odds processor (EOPP)](#5.2-Equalized-odds-processor-(EqOddsPr))
        * [5.2.1 EOPP on Logistic Regression](#5.2.1-Equalized-odds-processor-on-Logistic-Regression)
        * [5.2.2 EOPP on Random Forest](#5.2.2-Equalized-odds-processor-on-Random-Forest)

* [6. Summary of results](#6.-Summary-of-Model-Learning-Results)

* [7. Analyze rates of different mitigators](#7.-Analyze-rates-of-different-mitigators)

 -->

In [1]:
data_name = 'bank+marketing'

In [2]:
# to ignore warnings
import warnings
# warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
# tf.disable_eager_execution() # comment out when running TabTransformer models

## [1.](#Table-of-Contents) Prepare COMPAS data

In [4]:
# read in data as pandas data frame
df = pd.read_csv("../data/bank+marketing/bank/bank-full.csv", sep=';')
# check columns
pd.set_option('display.max_columns', None) # expand all columns
print(df.shape)
df.head()

(45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
# data pre-processing
import re
label = 'y'
protected_attribute = 'age'
# # 1. select features of interest
# keep_var = ['sex','age','race','juv_fel_count','juv_misd_count','juv_other_count','priors_count',
#             'c_charge_degree', 'two_year_recid']
# df = df[keep_var]

# 2. encode label and potential protected features 
df.age = df.age.apply(lambda x: 0 if x<=25 else 1)
df = df.reset_index(drop=True)

# 3. covert to binary variables if possible
binary_var = ['default', 'housing', 'loan', 'y']
binary_map = {'yes':1, 'no':0}
df[binary_var] = df[binary_var].replace(binary_map)

print(df.shape)
df.head()

(45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,0
1,1,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,0
2,1,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,0
3,1,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,0
4,1,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,0


In [6]:
# # flip labels to ensure pos_label = 1, not becoming recidivist
# df[label] = (~df[label].astype(bool)).astype(int)

# select descriptive features and target variable
X = df.drop(columns=label, axis=1) # select all features but target feature
y = df[[protected_attribute, label]] # include protected feature in order to AIF360

In [7]:
# set protected attribute as index
X = X.set_index([protected_attribute], append = True, drop = False)
y = y.set_index([protected_attribute], append = True)

# make y data frames to 1d array to pass modeling, but keep index (protected attribute)
y = pd.Series(y[label], index=y.index)

In [8]:
X.to_pickle('../data/{}/{}_X'.format(data_name, data_name))
y.to_pickle('../data/{}/{}_y'.format(data_name, data_name))

## [2.](#Table-of-Contents) Proposed GridSearch Approach 

The GridSearch Approach includes hyperparameter, threshold, and Bias Mitigation

In [9]:
from FairGridSearch import *
%load_ext autoreload
%autoreload 2
# allow automatic reloading of changes in FairGridSearch file

C:\Users\Andrew\Documents\study\Third\Ki1\khdv\Proj\Experiment\Fair_chuahoa\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pip install 'aif360[FairAdapt]'


In [10]:
# None, better
# 'RW', worse
# 'LFR_pre', stuck
# 'LFR_in', stuck
# 'AD', stuck
# 'EGR',better
# 'ROC', better
# 'CEO', better

## 3. Case Study: COMPAS

In [14]:
label = 'two_year_recid'
protected_attribute = 'age'

pos_label = 0
priv_group = 1

cv=10
n_jobs=multiprocessing.cpu_count()-1

In [15]:
X = pd.read_pickle('../data/{}/{}_X'.format(data_name, data_name))
y = pd.read_pickle('../data/{}/{}_y'.format(data_name, data_name))

In [16]:
# define desired metric for the use case
ACC_METRIC = 'avg_norm_mcc_score'
FAIR_METRIC = 'abs_avg_ppvd_score'

### Logistic Regression

In [ ]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'C':[1, 10],'solver':['liblinear', 'saga'],'penalty':['l2']}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','LFR_in','AD','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_lr = fair_GridsearchCV(base='LR', param_grid=param_grid, 
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_lr.fit(X=X, y=y)
    
results_lr = clf_lr.output_table
print(clf_lr._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_lr.to_pickle('./{}_results/{}_results_LR'.format(data_name,data_name))
style_table(results_lr)

  0%|          | 0/4 [00:00<?, ?it/s]

{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


### Random Forest

In [16]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'n_estimators':[10, 50],'criterion':['gini', 'entropy'], 'max_depth':[16]}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_rf = fair_GridsearchCV(base='RF', param_grid=param_grid,
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_rf.fit(X=X, y=y)
    
results_rf = clf_rf.output_table
print(clf_rf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_rf.to_pickle('./{}_results/{}_results_RF'.format(data_name, data_name))
style_table(results_rf)

  0%|          | 0/4 [00:00<?, ?it/s]

{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}
running Bias Mitigation None
running Bias Mitigation None
running Bias Mitigation None
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------
running Bias Mitigation RW
running Bias Mitigation RW
running Bias Mitigation RW
running Bias Mitigation LFR_pre
running Bias Mitigation LFR_prerunning Bias Mitigation LFR_pre

running Bias Mitigation EGR


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


running Bias Mitigation EGR


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \


...

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in D

running Bias Mitigation ROC
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
running Bias Mitigation CEO
running Bias Mitigation RW+ROC
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
running Bias Mitigation RW+CEO


100%|██████████| 4/4 [04:17<00:00, 64.34s/it]


base_estimator                                                    RF
param              {'criterion': 'entropy', 'max_depth': 16, 'n_e...
Bias_Mitigation                                               RW+CEO
threshold                                                        0.4
Name: 116, dtype: object
Time:  4.297946818283337 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.300000,0.615610,0.598383,0.703140,0.663190,0.230911,0.615455,0.106297,0.089824,0.141914,0.077496,0.155259,0.183870,0.110417,0.140535,0.490842
1,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.400000,0.627317,0.616621,0.689357,0.663190,0.246955,0.623477,0.119614,0.097943,0.145161,0.069217,0.140409,0.209138,0.138534,0.184877,0.496137
2,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.500000,0.624390,0.621109,0.655208,0.663190,0.243601,0.621800,0.163223,0.142134,0.196760,0.041868,0.141510,0.231089,0.179442,0.247596,0.541423
3,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.600000,0.610244,0.614493,0.601527,0.663190,0.229984,0.614992,0.171675,0.150868,0.195339,0.024559,0.134833,0.240195,0.230142,0.322105,0.556682
4,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",None,0.700000,0.578699,0.590770,0.510974,0.663190,0.193138,0.596569,0.154254,0.137963,0.172242,0.008952,0.134942,0.230504,0.300365,0.418587,0.557685
5,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.300000,0.610732,0.593396,0.699751,0.660284,0.219199,0.609599,0.062055,0.045177,0.090550,0.084772,0.175159,0.183480,0.111536,0.142540,0.452455
6,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.400000,0.620650,0.609560,0.685165,0.660284,0.233071,0.616535,0.074983,0.055433,0.110467,0.069537,0.175014,0.212000,0.139298,0.186063,0.458447
7,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.500000,0.619187,0.615167,0.653931,0.660284,0.232207,0.616104,0.089016,0.069172,0.129281,0.051353,0.169990,0.231740,0.177888,0.245035,0.472913
8,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.600000,0.608943,0.613295,0.599541,0.660284,0.227649,0.613824,0.096230,0.076152,0.132296,0.030064,0.158787,0.244715,0.231567,0.324046,0.482405
9,RF,"{'criterion': 'gini', 'max_depth': 16, 'n_estimators': 10}",RW,0.700000,0.585691,0.597399,0.522257,0.660284,0.206279,0.603139,0.091651,0.074004,0.118019,0.016628,0.147578,0.232065,0.292466,0.408987,0.488511


### Gradient Boosting

In [17]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'n_estimators':[10, 50],'criterion':['friedman_mse'],'max_depth':[8, 32]}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_gb = fair_GridsearchCV(base='GB', param_grid=param_grid,
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_gb.fit(X=X, y=y)
    
results_gb = clf_gb.output_table
print(clf_gb._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_gb.to_pickle('./{}_results/{}_results_GB'.format(data_name, data_name))
style_table(results_gb)

  0%|          | 0/4 [00:00<?, ?it/s]

{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}
running Bias Mitigation None
running Bias Mitigation None
running Bias Mitigation None
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------
running Bias Mitigation RW
running Bias Mitigation RW
running Bias Mitigation RW
running Bias Mitigation LFR_pre
running Bias Mitigation LFR_pre
running Bias Mitigation LFR_pre


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


running Bias Mitigation EGR


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \


...

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in D

running Bias Mitigation ROC
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
running Bias Mitigation CEO
running Bias Mitigation RW+ROC
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
0    385
1    230
Name: race, dtype: int64
running Bias Mitigation RW+CEO


100%|██████████| 4/4 [12:33<00:00, 188.45s/it]


base_estimator                                                    GB
param              {'criterion': 'friedman_mse', 'max_depth': 8, ...
Bias_Mitigation                                                   RW
threshold                                                        0.6
Name: 8, dtype: object
Time:  12.569811989583332 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.300000,0.585691,0.558000,0.713716,0.709030,0.205935,0.602967,0.078145,0.070057,0.112646,0.094617,0.084818,0.076846,0.071011,0.077126,0.475177
1,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.400000,0.639024,0.620677,0.725053,0.709030,0.290908,0.645454,0.164981,0.145556,0.212833,0.062511,0.081382,0.112228,0.096960,0.118003,0.519527
2,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.500000,0.660650,0.653442,0.705004,0.709030,0.315194,0.657597,0.214521,0.185615,0.237272,0.038121,0.084464,0.125528,0.141737,0.190566,0.556924
3,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.600000,0.635447,0.643286,0.606692,0.709030,0.292767,0.646383,0.151591,0.119631,0.128976,0.038681,0.104059,0.126081,0.233329,0.331968,0.505208
4,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",None,0.700000,0.534634,0.559683,0.302317,0.709030,0.174600,0.587300,0.125080,0.109205,0.079764,0.044276,0.100275,0.093041,0.424384,0.586527,0.537779
5,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.300000,0.586504,0.558784,0.714414,0.698300,0.207888,0.603944,0.045359,0.034228,0.054207,0.107467,0.074723,0.078764,0.070618,0.076438,0.441415
6,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.400000,0.633984,0.615036,0.723008,0.698300,0.281626,0.640813,0.084238,0.062670,0.109978,0.081647,0.141592,0.115545,0.095886,0.116354,0.443424
7,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.500000,0.653984,0.646434,0.700520,0.698300,0.301329,0.650665,0.069164,0.038242,0.072496,0.077299,0.140962,0.136911,0.142928,0.192255,0.418499
8,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.600000,0.631870,0.639058,0.607064,0.698300,0.283026,0.641513,0.000101,-0.030090,0.000332,0.077696,0.150672,0.122081,0.232145,0.329231,0.358588
9,GB,"{'criterion': 'friedman_mse', 'max_depth': 8, 'n_estimators': 10}",RW,0.700000,0.524715,0.550549,0.274185,0.698300,0.154687,0.577343,0.005671,-0.004980,0.010669,0.050535,0.130648,0.088650,0.441825,0.605819,0.428328


### Support Vector Machine

In [ ]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'kernel':['rbf','linear','poly','sigmoid'],'gamma':['scale']},
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_svm = fair_GridsearchCV(base='SVM', param_grid=param_grid,
                            prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                            cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_svm.fit(X=X, y=y)
    
results_svm = clf_svm.output_table
print(clf_svm._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_svm.to_pickle('./{}_results/{}_results_SVM'.format(data_name, data_name))
style_table(results_svm)

  0%|          | 0/4 [00:00<?, ?it/s]

{'gamma': 'scale', 'kernel': 'rbf'}
running Bias Mitigation None
running Bias Mitigation None
running Bias Mitigation None
------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------
running Bias Mitigation RW
running Bias Mitigation RW
running Bias Mitigation RW
running Bias Mitigation LFR_pre
running Bias Mitigation LFR_pre
running Bias Mitigation LFR_pre


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

running Bias Mitigation EGR


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_divi

...

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defi

running Bias Mitigation LFR_pre


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \


running Bias Mitigation EGR


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  lambda_event = (lambda_vec["+"] - self.ratio * lambda_vec["-"]).sum(level=_EVENT) / \
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/fairlearn/reductions/_moments/utility_parity.py:251: FutureWarning: Using the level keyword in D

### Gaussian Naive Bayes

In [ ]:
# model for Reweighing
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'var_smoothing': np.logspace(0,-9, num=4)}, 
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','LFR_pre','EGR','ROC','CEO','RW+ROC','RW+CEO']}

clf_nb = fair_GridsearchCV(base='NB', param_grid=param_grid, 
                           prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                           cv=cv, n_jobs=n_jobs)
if __name__ == '__main__':
    clf_nb.fit(X=X, y=y)
    
results_nb = clf_nb.output_table
# print(clf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_nb.to_pickle('./{}_results/{}_results_NB'.format(data_name, data_name))
style_table(results_nb)

### TabTransformer

In [15]:
import timeit
start = timeit.default_timer()

param_grid = {'hyperp_grid': {'epochs':[20, 30],'learing_rate':[1e-04, 1e-05]},
              'threshold': np.linspace(0.3, 0.7, 5),
              'Bias_Mitigation':[None,'RW','ROC','CEO','RW+ROC','RW+CEO']}

clf_tab = fair_GridsearchCV(base='TabTrans',param_grid=param_grid, 
                            prot_attr=protected_attribute, pos_label=pos_label, priv_group=priv_group,
                            cv=cv, n_jobs=n_jobs)

clf_tab.fit(X=X, y=y)
results_tab = clf_tab.output_table
# print(clf._best_param)

stop = timeit.default_timer()
runtime = stop - start
if runtime < 60:
    print('Time: ', runtime, 'sec')
else: print('Time: ', runtime/60, 'min')
results_tab.to_pickle('./{}_results/{}_results_TabTrans'.format(data_name, data_name))

style_table(results_tab)

  0%|          | 0/4 [00:00<?, ?it/s]

{'epochs': 20, 'learing_rate': 0.0001}



100%|██████████| 1/1 [00:00<00:00, 44.23it/s]
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
100%|██████████| 1/1 [00:00<00:00, 43.68it/s]
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future ver

------------------------------------------------------------------------------------------
start multiprocessing
------------------------------------------------------------------------------------------


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxi

...

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/tabtransformertf/utils/preprocessing.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  dataset[key] = value[:, tf.newaxis]

100%|██████████| 1/1 [00:00<00:00, 413.27it/s]


Epoch 1/30
9/9 [==============================] - 4s 140ms/step - loss: 0.6650 - PR AUC: 0.6376 - val_loss: 0.6485 - val_PR AUC: 0.6588
Epoch 2/30
9/9 [==============================] - 0s 45ms/step - loss: 0.6623 - PR AUC: 0.6368 - val_loss: 0.6487 - val_PR AUC: 0.6603
Epoch 3/30
9/9 [==============================] - 0s 45ms/step - loss: 0.6619 - PR AUC: 0.6280 - val_loss: 0.6488 - val_PR AUC: 0.6601
Epoch 4/30
9/9 [==============================] - 0s 45ms/step - loss: 0.6657 - PR AUC: 0.6276 - val_loss: 0.6487 - val_PR AUC: 0.6606
Epoch 5/30
9/9 [==============================] - 0s 46ms/step - loss: 0.6695 - PR AUC: 0.6194 - val_loss: 0.6484 - val_PR AUC: 0.6599
Epoch 6/30
9/9 [==============================] - 0s 46ms/step - loss: 0.6643 - PR AUC: 0.6281 - val_loss: 0.6482 - val_PR AUC: 0.6604
Epoch 7/30
9/9 [==============================] - 1s 71ms/step - loss: 0.6646 - PR AUC: 0.6261 - val_loss: 0.6483 - val_PR AUC: 0.6605
Epoch 8/30
9/9 [==============================] - 1s 7

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: False Omission Rate is ill-defined and being set to 0.0 due to no predicted negative samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 4/4 [1:04:57<00:00, 974.48s/it] 


Time:  64.97025443348333 min


,base_estimator,param,Bias_Mitigation,threshold,avg_acc_score,avg_bacc_score,avg_f1_score,avg_auc_score,avg_mcc_score,avg_norm_mcc_score,avg_spd_score,avg_aod_score,avg_eod_score,avg_ford_score,avg_ppvd_score,avg_(1-consistency_score),avg_gei_score,avg_ti_score,cost
0,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.300000,0.548293,0.516145,0.700947,0.664526,0.083032,0.541516,0.025913,0.023589,0.033285,0.114938,0.226717,0.011675,0.061514,0.063349,0.484397
1,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.400000,0.584065,0.559208,0.703620,0.664526,0.172788,0.586394,0.089418,0.080099,0.106684,0.096907,0.032715,0.037008,0.085041,0.100113,0.503024
2,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.500000,0.575447,0.568585,0.595157,0.664526,0.184049,0.592024,0.129427,0.115353,0.119468,0.080513,0.134910,0.056065,0.196894,0.263159,0.537403
3,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.600000,0.519675,0.534473,0.270793,0.664526,0.084310,0.542155,0.069266,0.062632,0.074072,-0.023069,0.098287,0.029951,0.401434,0.532748,0.527112
4,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",None,0.700000,0.501138,0.527355,0.161194,0.664526,0.056138,0.528069,0.043751,0.039316,0.052160,0.053382,0.125885,0.029236,0.484748,0.650644,0.515682
5,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.300000,0.584065,0.556623,0.711777,0.680423,0.198238,0.599119,0.085762,0.077531,0.114198,0.097660,0.005859,0.030732,0.072650,0.079971,0.486643
6,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.400000,0.623577,0.603138,0.719711,0.680423,0.262080,0.631040,0.156485,0.138368,0.182317,0.081641,0.007704,0.046829,0.092413,0.111030,0.525445
7,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.500000,0.649268,0.639828,0.702935,0.680423,0.294167,0.647084,0.215436,0.186528,0.209689,0.067236,0.025040,0.065659,0.135869,0.180901,0.568352
8,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.600000,0.576260,0.589937,0.463841,0.680423,0.201062,0.600531,0.169324,0.151253,0.160374,0.052330,0.107039,0.072813,0.322431,0.444481,0.568793
9,TabTrans,"{'epochs': 20, 'learing_rate': 0.0001}",RW,0.700000,0.485691,0.516361,0.099926,0.680423,0.052817,0.526408,0.046655,0.043100,0.038167,0.044526,0.116849,0.023447,0.527320,0.706378,0.520247
